In [64]:
import json
import pandas as pd
import ast

In [72]:
import pandas as pd
import ast
import numpy as np

# --- Configuration ---
INPUT_CSV_PATH = 'business_sample.csv'
OUTPUT_CSV_PATH = 'business_features_all_attributes.csv'
# --- End Configuration ---

# 1. Load the data
print(f"Loading data from: {INPUT_CSV_PATH}")
business_df = pd.read_csv(INPUT_CSV_PATH)

# Function to safely convert string dictionary from CSV to Python dict.
def safe_literal_eval(x):
    """Safely converts string dictionary from CSV to Python dict, handling complex escaping."""
    if pd.isna(x):
        return {}
    try:
        # Standardize quotes and handle unicode prefixes
        x = x.replace("u'", "'").replace("'", '"').replace('""', '"')
        # ast.literal_eval is generally safe and handles 'True'/'False' strings
        return ast.literal_eval(x)
    except (ValueError, SyntaxError, TypeError):
        # If parsing fails, return an empty dictionary
        return {}

# Function to safely convert the comma-separated category string to a Python list
def safe_categories_eval(x):
    try:
        if isinstance(x, str):
            return [c.strip() for c in x.split(',')]
        return []
    except Exception:
        return []

# --- Comprehensive Attribute Flattener Function ---
def flatten_and_extract_attributes(row):
    """Extracts and flattens ALL required attributes, standardizing to a clean format."""
    
    # 1. Start with core ID and the dictionary
    flat_attrs = {'business_id': row['business_id']}
    attributes = row['attributes_dict']
    
    # List of all simple (top-level) Boolean/Categorical attributes to extract
    SIMPLE_ATTRIBUTES = [
        'OutdoorSeating', 'RestaurantsDelivery', 'RestaurantsReservations', 'GoodForKids', 
        'RestaurantsGoodForGroups', 'RestaurantsTakeOut', 'WheelchairAccessible', 
        'RestaurantsTableService', 'GoodForDancing', 'HappyHour', 'CoatCheck', 
        'BusinessAcceptsCreditCards', 'DriveThr', 'ByAppointmentOnly', 'Smoking'
    ]
    
    # List of nested/complex attributes that need specialized handling
    COMPLEX_ATTRIBUTES = [
        'BusinessParking', 'Ambience', 'Music', 
        'WiFi', 'RestaurantsPriceRange2', 'NoiseLevel', 'Alcohol'
    ]

    # 2. Extract SIMPLE Attributes (Default is False or a controlled placeholder)
    for key in SIMPLE_ATTRIBUTES:
        value = attributes.get(key, False)
        # Convert True/False strings, 1/0 integers to a Python bool
        flat_attrs[key] = bool(value) 

    # 3. Extract COMPLEX/NESTED Attributes
    
    # 3.1. BusinessParking (Nested object)
    parking_dict = attributes.get('BusinessParking', {})
    if not isinstance(parking_dict, dict):
        parking_dict = {}
        
    for prefix in ['garage', 'street', 'validated', 'lot', 'valet']:
        col_name = f'{prefix}_parking'
        parking_value = parking_dict.get(prefix, False)
        flat_attrs[col_name] = bool(parking_value)

    # 3.2. Other Non-Boolean, Non-Nested attributes (Keep the value directly)
    # This covers categories like 'RestaurantsPriceRange2', 'NoiseLevel', 'Alcohol', 'WiFi'
    for key in ['RestaurantsPriceRange2', 'NoiseLevel', 'Alcohol', 'WiFi']:
        flat_attrs[key] = attributes.get(key, np.nan) 
    
    # Note: 'Ambience' and 'Music' are deep-nested and often require separate One-Hot Encoding
    # For now, we'll extract the main keys to keep the output manageable
    
    return flat_attrs

# 2. Select the base features
core_features = ['business_id', 'name', 'state', 'stars', 'review_count', 'categories', 'attributes']
features_df = business_df[core_features].copy()
features_df['attributes_dict'] = features_df['attributes'].apply(safe_literal_eval)

# --- 3. Process the 'attributes' column (Iteration-Based Extraction) ---
print("\nProcessing 'attributes' column (Full Extraction)...")

# Apply the custom function to flatten attributes and convert to a new DataFrame
attribute_data_list = features_df.apply(flatten_and_extract_attributes, axis=1).tolist()
df_attributes = pd.DataFrame(attribute_data_list)

# Merge the attributes back into the main DataFrame
features_df = pd.merge(features_df.drop(columns=['attributes', 'attributes_dict']), df_attributes, on='business_id', how='left')


# --- 4. Process the 'categories' column (Optimized One-Hot Encoding) ---
print("Processing 'categories' column (Optimized)...")

features_df['category_list'] = features_df['categories'].apply(safe_categories_eval)
all_categories = sorted(list(set(
    cat for sublist in features_df['category_list'] for cat in sublist
)))

category_columns = {}
for category in all_categories:
    col_name = f'Category_{category.replace(" ", "_").replace("&", "and")}'
    category_columns[col_name] = features_df['category_list'].apply(lambda x: 1 if category in x else 0)

categories_df_ohe = pd.DataFrame(category_columns)
features_df = pd.concat([features_df.reset_index(drop=True), categories_df_ohe.reset_index(drop=True)], axis=1)

# --- 5. Final cleanup and Save ---
features_df = features_df.drop(columns=['categories', 'category_list'])

print(f"\nSaving processed features to: {OUTPUT_CSV_PATH}")
features_df.to_csv(OUTPUT_CSV_PATH, index=False)

# 6. Display final result and column check
print("\n--- Processed DataFrame Head ---")

# Define columns for display
bool_attributes_display = ['RestaurantsTakeOut', 'OutdoorSeating', 'RestaurantsDelivery']
parking_display_cols = [col for col in features_df.columns if '_parking' in col]
category_display_cols = [col for col in features_df.columns if col.startswith('Category_')][:3]
other_attributes_display = ['RestaurantsPriceRange2', 'NoiseLevel']

final_cols = ['business_id', 'name', 'stars', 'review_count'] + bool_attributes_display + parking_display_cols + other_attributes_display + category_display_cols
print(features_df[final_cols].head())

print("\n--- Final Column Check ---")
print("Parking Columns:", [col for col in features_df.columns if '_parking' in col])
print(f"Total Features (Categories + Attributes + Core): {len(features_df.columns)}")
print(f"\nProcessing complete. Features saved to '{OUTPUT_CSV_PATH}'.")


# ... (rest of the code) ...

# 6. Display final result and column check
print("\n--- Processed DataFrame Head ---")
# ... (prints the head) ...

# ... (Previous code up to the print statement before the transposed inspection) ...

# -------------------------------------------------------------
## ⭐ DETAILED INSPECTION: SECOND BUSINESS (INDEX 1) ⭐
# -------------------------------------------------------------
print("\n\n#####################################################")
print("### DETAILED INSPECTION: SECOND BUSINESS (INDEX 1) ###")
print("#####################################################")

# Select the second row (index 1)
single_business_row = features_df.iloc[[1]]

# Redefine the function to INCLUDE all attribute columns
def filter_attributes_for_display_revised(index):
    # Get the value for the current column
    value = single_business_row.iloc[0].loc[index]
    
    # List of all attribute prefixes that should be shown (including simple attributes and parking)
    ATTRIBUTE_PREFIXES = [
        'OutdoorSeating', 'RestaurantsDelivery', 'RestaurantsReservations', 'GoodForKids', 
        'RestaurantsGoodForGroups', 'RestaurantsTakeOut', 'WheelchairAccessible', 
        'RestaurantsTableService', 'BusinessAcceptsCreditCards', 'garage_parking', 
        'street_parking', 'validated_parking', 'lot_parking', 'valet_parking'
    ]

    # 1. Keep core identifiers
    if index in ['business_id', 'name', 'state', 'stars', 'review_count']:
        return True
        
    # 2. Keep ALL explicitly extracted Boolean attributes (True OR False)
    if any(index.startswith(p) for p in ATTRIBUTE_PREFIXES) or index == 'TakeOut':
        return True
        
    # 3. Keep ONLY categories that are True (1)
    if index.startswith('Category_'):
        return value == 1
        
    return False

# Get the transposed and filtered result
readable_output = single_business_row.T.loc[single_business_row.T.index.to_series().apply(filter_attributes_for_display_revised)]

print(readable_output)
print(f"\nProcessing complete. Features saved to '{OUTPUT_CSV_PATH}'.")

print("\n--- Final Column Check ---")
print("Parking Columns:", [col for col in features_df.columns if '_parking' in col])
print(f"Total Features (Categories + Attributes + Core): {len(features_df.columns)}")

Loading data from: business_sample.csv

Processing 'attributes' column (Full Extraction)...
Processing 'categories' column (Optimized)...

Saving processed features to: business_features_all_attributes.csv

--- Processed DataFrame Head ---
              business_id                             name  stars  \
0  Vvq9QucD0IokLBlkbttd3Q                    Brioche Dorée    3.5   
1  qL4Ya3cBmPLIUlXdx0aEsw              Blue Taj Restaurant    2.5   
2  e0iCFeakO_y2Jwjm8stW8w                Taste of Szechuan    4.5   
3  J6KMedR-L-tP4wKIAsj7tw            Ricky's All Day Grill    2.5   
4  rPCdcIzXOeeAFfaUhdJJ6A  Original Joe's Restaurant & Bar    4.0   

   review_count  RestaurantsTakeOut  OutdoorSeating  RestaurantsDelivery  \
0             5               False           False                False   
1             7               False           False                False   
2            12               False           False                False   
3            23               False      